# MNIST CLASSIFICATION

### Imports

In [210]:
import tensorflow as tf

import deel.lipdp.layers as DP_layers
import deel.lipdp.losses as DP_losses
from deel.lipdp.pipeline import bound_clip_value
from deel.lipdp.pipeline import load_and_prepare_data
from deel.lipdp.sensitivity import get_max_epochs
from deel.lipdp.model import DP_Accountant
from deel.lipdp.model import DP_Sequential
from deel.lipdp.model import DPParameters
from deel.lipdp.model import AdaptiveLossGradientClipping

### Loading the data :

It is important to import the data with the right DP parameters to account properly for the privacy guarantees of the trained model.

In [211]:
ds_train, ds_test, dataset_metadata = load_and_prepare_data(
    "mnist",
    batch_size=2048,
    drop_remainder=True,  # accounting assumes fixed batch size
    bound_fct=bound_clip_value(
        10.0
    ),  # clipping preprocessing allows to control input bound
)

### Declaring the DP parameters :

We also need to declare explicitly the parameters of the DP training process.

In [212]:
dp_parameters = DPParameters(
    noisify_strategy="global",
    noise_multiplier=1.5,
    delta=1e-5,
)

### Defining the model :

We use a simple convolutive network to classify on the MNIST dataset. We add a loss gradient clipping layer at the end of our network for more tightness on our gradient's upper bound. Therefore allowing for better results with one less hyperparameter to tune for dynamically chosen clipping constant. 

In [213]:
layers = [
    DP_layers.DP_BoundedInput(
        input_shape=dataset_metadata.input_shape,
        upper_bound=dataset_metadata.max_norm,
    ),
    DP_layers.DP_SpectralConv2D(filters=16, kernel_size=5),
    DP_layers.DP_Flatten(),
    DP_layers.DP_SpectralDense(units=10),
    DP_layers.DP_ClipGradient(
        epsilon=1, mode="dynamic_svt", patience=10
    )
]

model = DP_Sequential(
    layers=layers, dp_parameters=dp_parameters, dataset_metadata=dataset_metadata
)

loss = DP_losses.DP_TauCategoricalCrossentropy(14.0)

# Compatible with any kind of non-private optimizer : 
opt = tf.keras.optimizers.SGD(learning_rate=1e-2)

model.compile(
    loss=loss,
    optimizer=opt,
    metrics=["accuracy"],
    run_eagerly=False,
)

/home/thomas.massena/Code/DEBUG/dp-lipschitz/lipdp_dev_env/lib/python3.9/site-packages/deel/lip/model.py:74: UserWarning: Sequential model contains a layer which is not a 1-Lipschitz layer: dp__bounded_input_39
  warn(_msg_not_lip.format(layer.name))
/home/thomas.massena/Code/DEBUG/dp-lipschitz/lipdp_dev_env/lib/python3.9/site-packages/deel/lip/model.py:74: UserWarning: Sequential model contains a layer which is not a 1-Lipschitz layer: dp__clip_gradient_39
  warn(_msg_not_lip.format(layer.name))


### Define the desired DP guarantees :

We compute the budget of epochs needed to yields the DP guarantees that you desire :

In [214]:
num_epochs = get_max_epochs(3.0, model)

epoch bounds = (0, 512.0) and epsilon = 57.32501154010554 at epoch 512.0
epoch bounds = (0, 256.0) and epsilon = 33.19136621177765 at epoch 256.0
epoch bounds = (0, 128.0) and epsilon = 18.700348347170372 at epoch 128.0
epoch bounds = (0, 64.0) and epsilon = 11.0321546467658 at epoch 64.0
epoch bounds = (0, 32.0) and epsilon = 6.628485024640014 at epoch 32.0
epoch bounds = (0, 16.0) and epsilon = 3.656193225171896 at epoch 16.0
epoch bounds = (8.0, 16.0) and epsilon = 1.9620813174159681 at epoch 8.0
epoch bounds = (8.0, 12.0) and epsilon = 3.309417344453968 at epoch 12.0
epoch bounds = (8.0, 10.0) and epsilon = 3.1263462214167053 at epoch 10.0
epoch bounds = (9.0, 10.0) and epsilon = 2.0637926226770054 at epoch 9.0


### Train the model : 

The training process is called through the model.fit attribute. We use the following callbacks : 

- **DP_Accountant** (log_fn) : accounts for the privacy guarantees after each epoch of training (*log_fn* makes it compatible with W&B logging).
- **DP_AdaptiveGradientClipping** (ds_train, patience) : automatically updates the losses's gradient clipping constant every *patience* steps. 


In [215]:
callbacks = [
    DP_Accountant(log_fn="logging"),
    AdaptiveLossGradientClipping(
        ds_train=ds_train
    ),  # DO NOT USE THIS CALLBACK WHEN mode != "dynamic_svt"
]

hist = model.fit(
    ds_train,
    epochs=num_epochs,
    validation_data=ds_test,
    callbacks=callbacks,
)

On train begin : 
Initial value is now equal to lipschitz constant of loss:  tf.Tensor(1.4142135, shape=(), dtype=float32)
Epoch 1/9


### 